In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [4]:
df = pd.read_csv("../data/raw/exploratory.csv")
df.head(5)

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark
0,1064,1064,F,1,FA,1,1/8/2020,0,0,9,9,ОСЕМЕН,236,7/1/2022,1H15132
1,1064,1064,F,1,FA,1,1/8/2020,0,0,9,9,НА_СХЕМУ,236,7/1/2022,9
2,1185,1185,F,1,FA,1,1/20/2020,0,49,9,43,ОСЕМЕН,213,7/1/2022,1H15184
3,1185,1185,F,1,FA,1,1/20/2020,0,49,9,43,НА_СХЕМУ,213,7/1/2022,9
4,1260,1260,F,1,FA,1,1/27/2020,0,49,5,5,НА_СХЕМУ,205,7/1/2022,9


# Предобработка

In [5]:
df['event_index'] = list(range(1, df.shape[0] + 1))

## Проверка на пустые значения

In [4]:
for column in df:
    print(column, ":", len(df.loc[(df[column].isnull()) | (df[column]=='-')].index))

id : 0
sid : 198
sex : 0
cycle : 0
childs : 5310
childbirth : 0
dob : 0
deadwood_days : 0
pregancy_days : 0
group_id : 0
group_past : 0
event : 0
milking_days : 0
date : 0
remark : 644
event_index : 0


В childs ставится NaN если ещё не было ребенка или животное мужского пола.
В remark стоит - если примечание не требуется

## Приведение к числовым значениям

In [145]:
df.dtypes

id                int64
sex              object
cycle             int64
childs           object
childbirth        int64
dob              object
deadwood_days     int64
pregancy_days     int64
group_id          int64
group_past        int64
event            object
milking_days      int64
date             object
remark           object
dtype: object

### Пол и дети

In [130]:
df['sex'] = df['sex'].apply(lambda x: 1 if x =='F' else 0)

In [131]:
df['childs_fa'] = df['childs'].apply(lambda x: str(x).count('FA'))
df['childs_ma'] = df['childs'].apply(lambda x: str(x).count('MA'))
df['childs_fd'] = df['childs'].apply(lambda x: str(x).count('FD'))
df['childs_md'] = df['childs'].apply(lambda x: str(x).count('MD'))

df['childs_m'] = df['childs'].apply(lambda x: str(x).count('M'))
df['childs_f'] = df['childs'].apply(lambda x: str(x).count('F'))
df['childs_d'] = df['childs'].apply(lambda x: str(x).count('D'))
df['childs_a'] = df['childs'].apply(lambda x: str(x).count('A'))


In [132]:
#df=df.drop(columns=["childs"])
df.dtypes

id                int64
sex               int64
cycle             int64
childs           object
childbirth        int64
dob              object
deadwood_days     int64
pregancy_days     int64
group_id          int64
group_past        int64
event            object
milking_days      int64
date             object
remark           object
childs_fa         int64
childs_ma         int64
childs_fd         int64
childs_md         int64
dtype: object

### Дата рождения

In [63]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['dob']  = pd.to_datetime(df['dob'], infer_datetime_format=True)
df['age']  = ((df['date'] - df['dob']) / np.timedelta64(1, 'D')).astype(int)

In [134]:
df=df.drop(columns=["dob"])
df.dtypes

id                        int64
sex                       int64
cycle                     int64
childs                   object
childbirth                int64
deadwood_days             int64
pregancy_days             int64
group_id                  int64
group_past                int64
event                    object
milking_days              int64
date             datetime64[ns]
remark                   object
childs_fa                 int64
childs_ma                 int64
childs_fd                 int64
childs_md                 int64
age                       int64
dtype: object

### Дата события

In [135]:
month=['jan','feb','mar','apr','may','jun', 'jul', 'aug', 'sep', 'oct', 'nov',' dec']
for m in range (12):
    df[month[m]]=df['date'].apply(lambda x: 1 if x.month == m + 1 else 0)
df.head()

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,1064,1,1,FA,1,0,0,9,9,ОСЕМЕН,...,0,0,0,0,1,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,НА_СХЕМУ,...,0,0,0,0,1,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,ОСЕМЕН,...,0,0,0,0,1,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,НА_СХЕМУ,...,0,0,0,0,1,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,НА_СХЕМУ,...,0,0,0,0,1,0,0,0,0,0


## Обработка событий

In [6]:
import re
from enum import IntEnum

class EventType(IntEnum):
    OTHER   = 0
    #-------------
    ABORT   = 101  # 
    TOSCM   = 102  #
    NULSCM  = 103  #
    DNB     = 104  #
    BRED    = 105  #
    FRESH   = 106  #
    PREG    = 107  #
    PREGBEF = 108  #
    DRY     = 109  #
    DRY2    = 110 #
    OPEN    = 111 #
    #--------------
    VAC     = 201 #
    VACVIR  = 202 #
    FOOTRIM = 203 #
    HEALTH  = 204 #
    BROKE   = 205 #
    POT     = 206 #
    ILLMISC = 207 #
    LAME    = 208 #
    KETOS   = 209 #
    MAST    = 210 #
    METR    = 211 #
    PARES   = 212 #
    RP      = 213 #
    #-------------
    WEIGHT  = 301 #
    DEAD    = 302 #
    MOVE    = 303 #
    SOLD    = 304 #
    
    def __str__(self):
        return self.name
    
    @staticmethod
    def from_str(label):
        EventNames = {
            EventType.ABORT   : ["ABORT", "АБОРТ"],  
            EventType.TOSCM   : ["TOSCM", "НА_СХЕМУ"], 
            EventType.NULSCM  : ["NULSCM","СО_СХЕМЫ"],
            EventType.DNB     : ["DNB", "НЕОСЕМ"],
            EventType.BRED    : ["BRED", "ОСЕМЕН"], 
            EventType.FRESH   : ["FRESH", "ОТЕЛ"],  
            EventType.PREG    : ["PREG", "СТЕЛН"], 
            EventType.PREGBEF : ["PREGBEF", "СТЕЛНДО"],
            EventType.DRY     : ["DRY", "СУХОСТ"],
            EventType.DRY2    : ["DRY2", "СУХ2"],
            EventType.OPEN    : ["OPEN", "ЯЛОВАЯ"],
            #-------
            EventType.WEIGHT  : ["WEIGHT", "ВЕС"],
            EventType.DEAD    : ["DEAD", "ПАЛА"],
            EventType.MOVE    : ["MOVE", "ПЕРЕВОД"],
            EventType.SOLD    : ["SOLD", "ПРОДАНА"],
            #-------
            EventType.VAC     : ["VAC", "ВАКЦИН"],
            EventType.VACVIR  : ["VACVIR", "ВАКВИРУС"],
            EventType.FOOTRIM : ["FOOTRIM", "РАСЧКОП"],
            EventType.HEALTH  : ["HEALTH", "WELL", "ЗДОРОВА"],
            EventType.BROKE   : ["BROKE", "ДЕФЕКТ"],
            EventType.POT     : ["POT", "ПРОФОТ"],
            EventType.ILLMISC : ["ILLMISC", "БОЛЕЗНЬ"],
            EventType.LAME    : ["LAME", "ХРОМОТА"],
            EventType.KETOS   : ["KETOS", "КЕТОЗ"],
            EventType.MAST    : ["MAST", "МАСТИТ"],
            EventType.METR    : ["METR", "МЕТРИТ"],
            EventType.PARES   : ["PARES", "ПАРЕЗ"],
            EventType.RP      : ["RP", "ПОСЛЕД"],
        }
        for ev in EventNames.keys():
            if label in EventNames[ev]:
                return ev
        return EventType(0)
    

In [7]:
df['event'] = df['event'].apply(lambda x: EventType.from_str(x))
df.head()

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index
0,1064,1064,F,1,FA,1,1/8/2020,0,0,9,9,105,236,7/1/2022,1H15132,1
1,1064,1064,F,1,FA,1,1/8/2020,0,0,9,9,102,236,7/1/2022,9,2
2,1185,1185,F,1,FA,1,1/20/2020,0,49,9,43,105,213,7/1/2022,1H15184,3
3,1185,1185,F,1,FA,1,1/20/2020,0,49,9,43,102,213,7/1/2022,9,4
4,1260,1260,F,1,FA,1,1/27/2020,0,49,5,5,102,205,7/1/2022,9,5


### Примечания к событиям
События делим на три группы:
1. События без примечаний
2. События, примечания к которым обрабатываются однообразно (например примечания --- номер протокола или id семени)
3. События, примечания к которым обрабытваются сложным образом.

Чтобы выбрать событие третьего типа, надо выбрать правило которым его надо обработать, а затем собственно произвести обработку.

In [66]:
for event in EventType:
    print(event, df[df['event'] == event]['remark'].unique().tolist())
    print('---------------------------------------------------------------------------')

OTHER []
---------------------------------------------------------------------------
ABORT ['94 ДНИ', '95 ДНИ', '81 ДНИ', 'БЕЗ ЖТ', '213 ДНИ', '172 ДНИ', 'ЖТ', 'ЛК', '241 ДНИ', '160 ДНИ', '83 ДНИ', '130 ДНИ', '84 ДНИ', '91 ДНИ', '7/25/2022', '214 ДНИ', 'БРАК', '54 ДНИ']
---------------------------------------------------------------------------
TOSCM ['9', '13', '40', '31', '29', '30', '38', '33', '46', '34', '36', '55', '60', '53', '37', '52', '42', '43', '58', '47', '48', '11', '2', '5', '15', '41', '49']
---------------------------------------------------------------------------
NULSCM ['-']
---------------------------------------------------------------------------
DNB ['ВЫМЯ', '-', 'МЕЛКАЯ', 'ВОСПР', 'НОГИ', 'ГИНЕКОЛ', 'НОГИПУТЫ', 'НАДОЙ', 'АТП', 'АБЦЕС', 'БРАК', 'СУСТАВ', 'ЗООТЕХ', 'ABC', 'МАСТИТ', 'ПНЕВМО', 'ЗДОРОВЬЕ']
---------------------------------------------------------------------------
BRED ['1H15132', '1H15184', '501H14018', '1H15689', '1H15461', '501H15515', '7H14578',

Некоторые замечания:
1. В примечаниях к событиям категории OPEN встречается 'БПАК' что судя по всему является опечаткой от 'БРАК'
2. В примечаниях к событиям категории OPEN встречается 'БЖТ' и 'БЕЗ ЖТ' и вероятно это одно и тоже
3. В примечаниях к событиям категории SOLD причина продажи и код причины продажи не находятся во взаимнооднозначном соответствии
4. Примечания к событиям LAME и MAST не единобразны
5. В примечании к событии ILLMISC есть протокол БУР1_?

**Что делаем:**

Фиксим пункты 1 и 2, в примечаниях к событиям SOLD и DEAD оставляем только причину продажи/смерти (без кода). 

Примечания к событиям MAST и LAME приводим к виду {Код протокола}_{пораженные соски/ноги}

In [8]:
df['remark'] = df.apply(lambda row: 'БРАК' if (row.event == EventType.OPEN and row.remark == 'БПАК') else row.remark, axis=1)
df['remark'] = df.apply(lambda row: 'БЕЗ ЖТ' if  (row.event == EventType.OPEN and row.remark == 'БЖТ') else row.remark, axis=1)
df['remark'] = df.apply(lambda row: row.remark.split(';')[1].strip().rstrip() if row.event == EventType.SOLD else row.remark, axis=1)
df['remark'] = df.apply(lambda row: row.remark.split(';')[1].strip().rstrip() if row.event == EventType.DEAD else row.remark, axis=1)


def get_protocol_id(s):
    return s[0:s.find('_'):]

def get_protocol_num(s):
    m = []
    t = s[s.find('_') + 1::]
    for i in range(len(t)):
        if t[i] == '_':
            for j in range(int(t[i - 1])+1, int(t[i + 1])):
                m.append(j)
        elif t[i].isdigit():
            m.append(int(t[i]))
    return m

df['remark'] = df.apply(lambda row: get_protocol_id(row.remark) + '_' + ''.join(list(map(str, get_protocol_num(row.remark)))) if (row.event == EventType.LAME or row.event == EventType.MAST) else row.remark, axis=1)

#### Механика парсинга:
К каждому событию составляется набор правил:

Правило это функция + регулярное выражение. Если примечание удовлетворяет регулярке, то его содержимое отправляется в функцию, а выход функции записывается в каунтер {ИМЯ СОБЫТИЯ}_{НОМЕР ПРАВИЛА}

In [10]:
class EventParsingRule:
    def __init__(self, pattern, parser):
        self.raw = pattern
        self.parse = parser
        self.pattern = re.compile(pattern)
    
        
RuleBin  = EventParsingRule('.*', lambda x: 1)
RuleNum  = EventParsingRule('[0-9]*', lambda x: int(x)) 
RuleDays = EventParsingRule('[0-9]* ДНИ', lambda x: int(x[0:-4]))

def satisfy(s, rules):
    for rule in rules:
        if rule.pattern.match(s):
            return True
    return False

def generate_unique_rules(event):
    global df
    patterns = df[df['event'] == event]['remark'].unique().tolist()
    rules = []
    for p in patterns:
        rules.append(EventParsingRule(p, lambda x: 1))
    
    return rules

def generate_combined_rules(specific_rules, event):
    global df
    rules = specific_rules.copy()
    patterns = df[df['event'] == event].loc[df['remark'].apply(lambda s: satisfy(s, specific_rules)) == False]['remark'].unique().tolist()
    for p in patterns:
        rules.append(EventParsingRule(p, lambda x: 1))
    
    return rules


def generate_protocol_rules(event):
    global df
    protocols = set(map(get_protocol_id, df[df['event'] == event]['remark'].unique().tolist()))
    rules = []
    for p in protocols:
        for i in range(1, 5):
            rules.append(EventParsingRule(p + '_' + f'(([0-9]+{i})|{i})[0-9]*', lambda x: 1))
    return rules

EventRules = {
    EventType.OTHER   : [RuleBin],
    EventType.ABORT   : generate_combined_rules([RuleDays], EventType.ABORT),
    EventType.TOSCM   : generate_unique_rules(EventType.TOSCM), 
    EventType.NULSCM  : [RuleBin],
    EventType.DNB     : generate_unique_rules(EventType.DNB), 
    EventType.BRED    : generate_unique_rules(EventType.BRED), 
    EventType.FRESH   : [RuleBin],  
    EventType.PREG    : [RuleDays], 
    EventType.PREGBEF : [RuleDays],
    EventType.DRY     : generate_unique_rules(EventType.DRY), 
    EventType.DRY2    : [RuleBin],
    EventType.OPEN    : generate_unique_rules(EventType.OPEN),
    #-------
    EventType.WEIGHT  : [RuleNum],
    EventType.DEAD    : generate_unique_rules(EventType.DEAD),
    EventType.MOVE    : [RuleBin],
    EventType.SOLD    : generate_unique_rules(EventType.SOLD),
    #-------
    EventType.VAC     : generate_unique_rules(EventType.VAC),
    EventType.VACVIR  : generate_unique_rules(EventType.VACVIR),
    EventType.FOOTRIM : generate_unique_rules(EventType.FOOTRIM), # Может быть тут Bin поставить?
    EventType.HEALTH  : [RuleBin],
    EventType.BROKE   : generate_unique_rules(EventType.BROKE),
    EventType.POT     : generate_unique_rules(EventType.POT),
    EventType.ILLMISC : generate_unique_rules(EventType.ILLMISC),
    EventType.LAME    : generate_protocol_rules(EventType.LAME),
    EventType.KETOS   : generate_unique_rules(EventType.KETOS),
    EventType.MAST    : generate_protocol_rules(EventType.MAST),
    EventType.METR    : generate_unique_rules(EventType.METR),
    EventType.PARES   : generate_unique_rules(EventType.PARES),
    EventType.RP      : generate_unique_rules(EventType.RP),
}

#Get All Rules:
print("Правила парсинга:")
for event in EventType:
    print(str(event)+':')
    for number in range(len(EventRules[event])):
        print(str(event)+'_'+str(number) + ' :', EventRules[event][number].raw)
    print('------')

def ParseRemark(event, remark):
    global EventRules
    for rule in EventRules[event]:
        if rule.pattern.match(remark):
            return (rule.parse(remark))
    return 0

for event in EventType:
    df[str(event)] =  df.apply(lambda row: 1 if row.event == event else 0, axis=1)
    for number in range(len(EventRules[event])):
        df[str(event)+'_'+str(number)] = df.apply(lambda row: ParseRemark(event, row.remark) if row.event == event else 0, axis=1)

Правила парсинга:
OTHER:
OTHER_0 : .*
------
ABORT:
ABORT_0 : [0-9]* ДНИ
ABORT_1 : БЕЗ ЖТ
ABORT_2 : ЖТ
ABORT_3 : ЛК
ABORT_4 : 7/25/2022
ABORT_5 : БРАК
------
TOSCM:
TOSCM_0 : 9
TOSCM_1 : 13
TOSCM_2 : 40
TOSCM_3 : 31
TOSCM_4 : 29
TOSCM_5 : 30
TOSCM_6 : 38
TOSCM_7 : 33
TOSCM_8 : 46
TOSCM_9 : 34
TOSCM_10 : 36
TOSCM_11 : 55
TOSCM_12 : 60
TOSCM_13 : 53
TOSCM_14 : 37
TOSCM_15 : 52
TOSCM_16 : 42
TOSCM_17 : 43
TOSCM_18 : 58
TOSCM_19 : 47
TOSCM_20 : 48
TOSCM_21 : 11
TOSCM_22 : 2
TOSCM_23 : 5
TOSCM_24 : 15
TOSCM_25 : 41
TOSCM_26 : 49
------
NULSCM:
NULSCM_0 : .*
------
DNB:
DNB_0 : ВЫМЯ
DNB_1 : -
DNB_2 : МЕЛКАЯ
DNB_3 : ВОСПР
DNB_4 : НОГИ
DNB_5 : ГИНЕКОЛ
DNB_6 : НОГИПУТЫ
DNB_7 : НАДОЙ
DNB_8 : АТП
DNB_9 : АБЦЕС
DNB_10 : БРАК
DNB_11 : СУСТАВ
DNB_12 : ЗООТЕХ
DNB_13 : ABC
DNB_14 : МАСТИТ
DNB_15 : ПНЕВМО
DNB_16 : ЗДОРОВЬЕ
------
BRED:
BRED_0 : 1H15132
BRED_1 : 1H15184
BRED_2 : 501H14018
BRED_3 : 1H15689
BRED_4 : 1H15461
BRED_5 : 501H15515
BRED_6 : 7H14578
BRED_7 : 501H15681
BRED_8 : 614H15481
BRED_9 :

Посмотрим что получилось:

In [141]:
df.head(5)

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,SOLD_2,SOLD_3,SOLD_4,SOLD_5,SOLD_6,SOLD_7,SOLD_8,SOLD_9,SOLD_10,SOLD_11
0,1064,1,1,FA,1,0,0,9,9,105,...,0,0,0,0,0,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,102,...,0,0,0,0,0,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,105,...,0,0,0,0,0,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,102,...,0,0,0,0,0,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,102,...,0,0,0,0,0,0,0,0,0,0


## Учёт истории и переводы
Теперь мы переходим к обработке контексто-зависимых забытий. Для этого надо иметь понимаение какое событие произошло ДО текущего события и ПОСЛЕ текущего события.

Как понять очередность событий, если они происходят в один день? Для начала надо посмотреть на примеры такого.

In [161]:
sf = df.groupby(['id', 'date']).count()
sf[sf['event'] > 3]

,,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,remark
id,date,,,,,,,,,,,,
4146,7/3/2022,4,4,4,4,4,4,4,4,4,4,4,4
4245,7/29/2022,4,4,4,4,4,4,4,4,4,4,4,4
6214,7/1/2022,4,4,4,4,4,4,4,4,4,4,4,4
8036,7/29/2022,4,4,4,4,4,4,4,4,4,4,4,4
11293,7/20/2022,4,4,0,4,4,4,4,4,4,4,4,4
12171,7/20/2022,4,4,0,4,4,4,4,4,4,4,4,4
20398,7/21/2022,4,4,0,4,4,4,4,4,4,4,4,4
20855,7/12/2022,4,4,0,4,4,4,4,4,4,4,4,4
803063,7/8/2022,4,4,4,4,4,4,4,4,4,4,4,4


Иногда (но очень редко), с коровой происходит несколько событий в один день. Как правило эти события отделены переводом, а между переводами порядок не важен.

Кстати параметр "Предыдущая группа животного" указывает на группу в предыдущем событии! Сортировка событий в течении дня происходит так:
1. Разбить события на на группы (каждая группа отвечает что делали с животным когда оно было в соотвествующей группе)
2. Упорядочить группы в соответсвии с информацией о перводах
3. Упорядочить события внутри группы по порядку из Enuma

### События перемещения:
Надо проверить, что события перемещения отображают корректную информацию

In [111]:
df['incorrect_move'] = df.apply(lambda row: 1 if row.event == EventType.MOVE and int(row.remark[1:4]) != row.group_id else 0, axis=1)
df[df['incorrect_move'] == 1].head()

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,incorrect_move


### Сортировка событий:

**TODO**:
Написано максимально криво надо подправить. К тому же есть вопросы к эффективности

In [9]:
print(df[df['event'] == EventType.FRESH]['group_id'].unique().tolist())

cond = ((df['event'] == EventType.FRESH) & (df['group_id'] == 18))
df[cond].head(10)

[18, 17, 45, 33, 40]


,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index
126,8136,8136,F,1,FA,1,8/17/2020,0,0,18,8,106,0,7/1/2022,-,127
353,612062,612062,F,4,MA,1,12/12/2016,63,0,18,21,106,0,7/1/2022,-,354
432,811180,811180,F,3,MA,1,11/24/2018,57,0,18,10,106,0,7/1/2022,-,433
475,907168,907168,F,2,FA,1,7/24/2019,64,0,18,4,106,0,7/1/2022,-,476
480,907225,907225,F,2,FA,1,7/30/2019,71,0,18,10,106,0,7/1/2022,-,481
487,908049,908049,F,2,MA,1,8/9/2019,49,0,18,43,106,0,7/1/2022,-,488
502,909158,909158,F,2,FA,1,9/21/2019,57,0,18,10,106,0,7/1/2022,-,503
512,910131,910131,F,2,FA,1,10/19/2019,57,0,18,10,106,0,7/1/2022,-,513
514,910171,910171,F,2,FA,1,10/26/2019,64,0,18,10,106,0,7/1/2022,-,515
604,807075,807075,F,3,FA,1,7/14/2018,58,0,18,2,106,0,7/2/2022,-,605


In [12]:
import itertools
def path_sort(edge_list):
    m = len(edge_list)
    if m == 1:
        return edge_list
    
    for sigma in itertools.permutations(list(range(m))):
        bad=False
        for i in range(1, m):
            if edge_list[sigma[i]][1] != edge_list[sigma[i-1]][0]:
                bad=True
                break
        if not bad:
            edge_list_fixed = [edge_list[sigma[i]] for i in range(1, m)]
            return edge_list_fixed
    return edge_list


def get_day_ordering(r):
    
    cow_id   = r.id
    ev_date  = r.date 
    ev_index = r.event_index
    
    global df
    today = df[(df['id'] == cow_id) & (df['date'] == ev_date)].copy()
    
    moves = today[today['event'] == EventType.MOVE]
    if moves.shape[0] == 0:
        today.sort_values(by=['event'])
        today['ord'] = list(range(1, today.shape[0] + 1, 1))
        return (today[today['event_index'] == ev_index]['ord'].iloc[0])
   
    moves['to']   = moves['remark'].apply(lambda x: int(x[5::]))
    moves['from'] = moves['remark'].apply(lambda x: int(x[1:4:]))
    
    edge_list  = path_sort(moves.apply(lambda row: (row['from'], row['to']), axis=1).tolist())
    path = [edge[0] for edge in edge_list]
    path.append(edge_list[-1][1])
    
    try:
        today['group_ord'] = today['group_id'].apply(lambda x: path.index(x))
        today.sort_values(by=['group_ord', 'event'])
    except:
        #print('------------------')
        #print(edge_list)
        #print(row)
        #print('------------------')
        today.sort_values(by=['event'])
        
    today['ord'] = list(range(1, today.shape[0] + 1, 1))
    
    return (today[today['event_index'] == ev_index]['ord'].iloc[0])

In [13]:
df['ord'] = df.apply(lambda row: get_day_ordering(row), axis=1)
df.sort_values(by=['id', 'date', 'ord'])

------------------
[(44, 21)]
id                    8136
sid                   8136
sex                      F
cycle                    1
childs                  FA
childbirth               1
dob              8/17/2020
deadwood_days            0
pregancy_days            0
group_id                44
group_past               8
event                  303
milking_days             0
date              7/1/2022
remark            F044T021
event_index            126
ord                      1
Name: 125, dtype: object
------------------
------------------
[(44, 21)]
id                    8136
sid                   8136
sex                      F
cycle                    1
childs                  FA
childbirth               1
dob              8/17/2020
deadwood_days            0
pregancy_days            0
group_id                18
group_past               8
event                  106
milking_days             0
date              7/1/2022
remark                   -
event_index            127
ord  

------------------
[(5, 12)]
id                 702036
sid                702036
sex                     F
cycle                   4
childs                 FA
childbirth              1
dob              2/5/2017
deadwood_days          52
pregancy_days         158
group_id                1
group_past             12
event                 207
milking_days          205
date             7/5/2022
remark             СУСТАВ
event_index          1533
ord                     1
Name: 1532, dtype: object
------------------
------------------
[(5, 12)]
id                 702036
sid                702036
sex                     F
cycle                   4
childs                 FA
childbirth              1
dob              2/5/2017
deadwood_days          52
pregancy_days         158
group_id               12
group_past             12
event                 204
milking_days          205
date             7/5/2022
remark            ХРОМОТА
event_index          1534
ord                     2
Name: 1533, d

------------------
[(17, 45)]
id                  709076
sid                 709076
sex                      F
cycle                    3
childs                  MA
childbirth               1
dob              9/16/2017
deadwood_days           63
pregancy_days          215
group_id                42
group_past              17
event                  109
milking_days           343
date              7/6/2022
remark                   -
event_index           1869
ord                      1
Name: 1868, dtype: object
------------------
------------------
[(17, 45)]
id                  709076
sid                 709076
sex                      F
cycle                    3
childs                  MA
childbirth               1
dob              9/16/2017
deadwood_days           63
pregancy_days          215
group_id                17
group_past              17
event                  303
milking_days           343
date              7/6/2022
remark            F017T045
event_index           1870
ord 

------------------
[(6, 12)]
id                    8036
sid                   8036
sex                      F
cycle                    1
childs                  FD
childbirth               2
dob               8/6/2020
deadwood_days            0
pregancy_days            0
group_id                 6
group_past               8
event                  105
milking_days            59
date             7/29/2022
remark           614H15481
event_index           4489
ord                      1
Name: 4488, dtype: object
------------------
------------------
[(6, 12)]
id                    8036
sid                   8036
sex                      F
cycle                    1
childs                  FD
childbirth               2
dob               8/6/2020
deadwood_days            0
pregancy_days            0
group_id                 9
group_past               8
event                  102
milking_days            59
date             7/29/2022
remark                  13
event_index           4490
ord   

------------------
[(44, 21)]
id                  606021
sid                 606021
sex                      F
cycle                    5
childs                  MA
childbirth               1
dob               6/4/2016
deadwood_days           57
pregancy_days            0
group_id                18
group_past              21
event                  106
milking_days             0
date             7/22/2022
remark                   -
event_index           9103
ord                      1
Name: 9102, dtype: object
------------------
------------------
[(44, 21)]
id                  606021
sid                 606021
sex                      F
cycle                    5
childs                  MA
childbirth               1
dob               6/4/2016
deadwood_days           57
pregancy_days            0
group_id                44
group_past              21
event                  213
milking_days             0
date             7/22/2022
remark             ПОСЛЕД2
event_index           9104
ord 

------------------
[(6, 12)]
id                  811042
sid                 811042
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              11/6/2018
deadwood_days           66
pregancy_days          230
group_id                 6
group_past               7
event                  303
milking_days           289
date             7/19/2022
remark            F006T012
event_index          10044
ord                      1
Name: 10043, dtype: object
------------------
------------------
[(6, 12)]
id                  811042
sid                 811042
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              11/6/2018
deadwood_days           66
pregancy_days          230
group_id                12
group_past               7
event                  204
milking_days           289
date             7/19/2022
remark             ХРОМОТА
event_index          10045
ord  

------------------
[(44, 21)]
id                  910033
sid                 910033
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              10/5/2019
deadwood_days           56
pregancy_days            0
group_id                44
group_past              21
event                  303
milking_days             0
date             7/28/2022
remark            F044T021
event_index          11055
ord                      1
Name: 11054, dtype: object
------------------
------------------
[(44, 21)]
id                  910033
sid                 910033
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              10/5/2019
deadwood_days           56
pregancy_days            0
group_id                44
group_past              21
event                  206
milking_days             0
date             7/28/2022
remark               ПРОФ2
event_index          11056
ord

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
1965,1001,1001,F,1,FA,1,1/1/2020,0,203,9,9,203,248,7/7/2022,КАН,1966,1
3805,1005,1005,F,1,MA,1,1/1/2020,0,173,6,6,303,254,7/13/2022,F006T007,3806,1
2881,1007,1007,F,1,FA,1,1/1/2020,0,256,5,17,303,268,7/11/2022,F005T006,2882,1
3806,1007,1007,F,1,FA,1,1/1/2020,0,258,6,17,107,275,7/18/2022,214 ДНИ,3807,1
3807,1007,1007,F,1,FA,1,1/1/2020,0,258,6,17,109,278,7/21/2022,CEBA,3808,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11554,912269,912269,F,1,FA,1,12/31/2019,0,223,7,7,303,242,7/19/2022,F007T012,11555,1
11555,912269,912269,F,1,FA,1,12/31/2019,0,223,12,7,204,242,7/19/2022,ХРОМОТА,11556,2
11556,912270,912270,F,1,MA,1,12/31/2019,0,257,7,17,203,275,7/16/2022,КАН,11557,1
11557,912270,912270,F,1,MA,1,12/31/2019,0,257,7,17,107,277,7/18/2022,213 ДНИ,11558,1


**TODO** Спросить что вообще с номерами групп происходит:

In [15]:
df[((df['date'] == '7/29/2022') & (df['id'] == 8036))].head(10)

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
4488,8036,8036,F,1,FD,2,8/6/2020,0,0,6,8,105,59,7/29/2022,614H15481,4489,1
4489,8036,8036,F,1,FD,2,8/6/2020,0,0,9,8,102,59,7/29/2022,13,4490,2
4490,8036,8036,F,1,FD,2,8/6/2020,0,0,6,8,303,59,7/29/2022,F006T012,4491,3
4491,8036,8036,F,1,FD,2,8/6/2020,0,0,12,8,204,59,7/29/2022,ХРОМОТА,4492,4


In [16]:
df[((df['date'] == '7/22/2022') & (df['id'] == 807163))].head(10)

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
9797,807163,807163,F,3,FA,1,7/28/2018,106,0,44,2,303,0,7/22/2022,F044T021,9798,1
9798,807163,807163,F,3,FA,1,7/28/2018,106,0,18,2,106,0,7/22/2022,-,9799,2


На самом деле такое решение имеет смысл только если нет ситуации когда животное кидают из группы в группу по циклу (например перевели из 3 в 12 потом обратно в 3), в таком случае восстановить порядок наверняка невозможно. Но судя по всему такого не происходит

###  Cобытия выздоровления и болезней:
Заболевания КЕТОЗ, МЕТРИТ, ПАРЕЗ, ПОСЛЕД, ПРОФОТ протекают в новотельный период и имеют общий критерий выписки: все послеотельные осложнения животного вылечили, корова пригодна к дальнейшей работе и воспроизводству (примечание при выписке ГИНЕКОЛ). Примечание МОЛОДНЯК к событию ЗДОРОВА имеют только животные 0-й лактации (телки и бычки). Примечание ПРОЧИЕ к событию ЗДОРОВА имеют животные, перенесшие прочие заболевания (событие БОЛЕЗНЬ).

In [42]:
df.head()

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,MAST_КМ6_3,MAST_КМ6_4,MAST_ТМ1_1,MAST_ТМ1_2,MAST_ТМ1_3,MAST_ТМ1_4,MAST_КМ3_1,MAST_КМ3_2,MAST_КМ3_3,MAST_КМ3_4
0,1064,1,1,FA,1,0,0,9,9,BRED,...,0,0,0,0,0,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,TOSCM,...,0,0,0,0,0,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,BRED,...,0,0,0,0,0,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,TOSCM,...,0,0,0,0,0,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,TOSCM,...,0,0,0,0,0,0,0,0,0,0


# Генерация новых фичей